In [1]:
# Import libraries and dependencies
import numpy as np
import pandas as pd

# Access API keys
import os
from dotenv import load_dotenv

D:\Users\ogo\Anaconda3\envs\pyvizenv2\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
# Import the main functionality from the SimFin API
import simfin as sf

# Import names used for easy access to SimFin's data-columns
from simfin.names import *

# Version of the SimFin Python API
sf.__version__

'0.8.1'

In [3]:
# Set file directory for data from SimFin API
sf.set_data_dir('~/simfin_data/')

In [4]:
load_dotenv()
SIMFIN_KEY = os.getenv("SIMFIN_KEY") 
SIMKEY1 = '''Must have SIMFIN+ API to get most recent data, use free API for older data to run notebook'''
len(SIMFIN_KEY), SIMFIN_KEY

In [5]:
sf.set_api_key(SIMKEY1)

In [6]:
%%time

# Data for USA.
market = 'us'

hub = sf.StockHub(market=market,
                  refresh_days=1,
                  refresh_days_shareprices=1)

# Quarterly Income Statements.
income_qtr = sf.load_income(variant='quarterly-full', market=market)

# Quarterly Balance Sheets.
balance_qtr = sf.load_balance(variant='quarterly-full', market=market)

# Quarterly Cash-Flow Statements.
cashflow_qtr = sf.load_cashflow(variant='quarterly-full', market=market)

# Download US market financial data history
financial_data = hub.fin_signals(variant='latest')
derived_data = sf.load_derived(variant='quarterly', market='us')

# Daily Share-Prices.
historical_prices = sf.load_shareprices(variant='daily', market=market)
current_prices = sf.load_shareprices(variant='latest', market=market)

Dataset "us-income-quarterly-full" on disk (7 days old).
- Loading from disk ... Done!
Dataset "us-balance-quarterly-full" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-quarterly-full" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-income-ttm" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-balance-ttm" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-ttm" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-shareprices-latest" on disk (0 days old).
- Loading from disk ... Done!
Cache-file 'fin_signals-899db0b4.pickle' on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-derived-quarterly" on disk (7 days old).
- Loading from disk ... Done!
Dataset "us-shareprices-daily" on disk (7 days old).
- Loading from disk ... Done!
Dataset "us-shareprices-latest" on disk (0 days old).
- Loading from disk ... Done!
Wall time: 1min 3s


In [7]:
# Initialize list of 30 component stocks from Dow Jones Industrial Average

dow30 = [ "AAPL","AMGN","AXP","BA","CAT","CRM",'CSCO','CVX','DIS','DOW',
         'GS','HD','HON','IBM','INTC','JNJ','JPM','KO','MCD','MMM','MRK',
         'MSFT','NKE','PG','TRV','UNH','V','VZ','WBA',"WMT"]

In [8]:
dow30_px_history = historical_prices.loc[dow30, [CLOSE]].copy()
dow30_px_history[:2]

Close
Ticker Date             
AAPL   2007-01-03   2.99
       2007-01-04   3.06

In [9]:
dow30_px_latest = current_prices.loc[dow30, [CLOSE]].copy()
dow30_px_latest[:2]

,,Close
Ticker,Date,
AAPL,2020-10-16,119.02
AMGN,2020-10-16,235.72


In [11]:
# Create data subset from Dow 30 component stocks

dow30_income_qtr = income_qtr.loc[dow30].copy()
dow30_balance_qtr = balance_qtr.loc[dow30].copy()
dow30_cashflow_qtr = cashflow_qtr.loc[dow30].copy()

dow30_ratios_data = derived_data.loc[dow30].copy()

In [12]:
dow30_income_qtr.columns, dow30_balance_qtr.columns, dow30_cashflow_qtr.columns

(Index(['SimFinId', 'Currency', 'Fiscal Year', 'Fiscal Period', 'Publish Date',
        'Restated Date', 'Shares (Basic)', 'Shares (Diluted)', 'Source',
        'Revenue', 'Sales & Services Revenue', 'Financing Revenue',
        'Other Revenue', 'Cost of Revenue', 'Cost of Goods & Services',
        'Cost of Financing Revenue', 'Cost of Other Revenue', 'Gross Profit',
        'Other Operating Income', 'Operating Expenses',
        'Selling, General & Administrative', 'Selling & Marketing',
        'General & Administrative', 'Research & Development',
        'Depreciation & Amortization', 'Provision for Doubtful Accounts',
        'Other Operating Expenses', 'Operating Income (Loss)',
        'Non-Operating Income (Loss)', 'Interest Expense, Net',
        'Interest Expense', 'Interest Income', 'Other Investment Income (Loss)',
        'Foreign Exchange Gain (Loss)', 'Income (Loss) from Affiliates',
        'Other Non-Operating Income (Loss)', 'Pretax Income (Loss), Adj.',
        'Abno

In [13]:
df1 = dow30_income_qtr['Operating Income (Loss)'].fillna(0).copy()
df2 = dow30_income_qtr['Revenue'].fillna(0).copy()
df3 = dow30_income_qtr['Net Income'].fillna(0).copy()
df4 = dow30_balance_qtr['Total Liabilities'].fillna(0).copy()
df5 = dow30_balance_qtr['Total Equity'].fillna(0).copy()
dow30_fundamentals = pd.concat([df1, df2, df3, df4, df5], axis=1)
dow30_fundamentals[:2]

Operating Income (Loss)       Revenue  Net Income  \
Ticker Report Date                                                      
AAPL   2000-06-30               168000000.0  1.825000e+09   200000000   
       2000-09-30                84000000.0  1.870000e+09   170000000   

                    Total Liabilities  Total Equity  
Ticker Report Date                                   
AAPL   2000-06-30          2696000000  4.107000e+09  
       2000-09-30          2696000000  4.107000e+09

In [14]:
# Utilize shift to get mean of current and previous quarter total equity
dow30_equity_mean = 0.5 * (dow30_fundamentals['Total Equity'] + dow30_fundamentals['Total Equity'].shift(1))

# Create calc target data 

dow30_fundamentals['ROE'] = dow30_fundamentals['Net Income'] / dow30_equity_mean

dow30_fundamentals['DebtEquity'] =  dow30_fundamentals['Total Liabilities'] / dow30_equity_mean

dow30_fundamentals['OpMargin'] = dow30_fundamentals['Operating Income (Loss)'] / dow30_fundamentals['Revenue']

In [15]:
# Add F-score to dataset

dow30_fundamentals['Fscore'] = dow30_ratios_data['Pietroski F-Score'].fillna(0).copy()
dow30_financial_data = dow30_fundamentals.drop(columns = ['Operating Income (Loss)', 'Revenue', 
                                                          'Net Income', 'Total Liabilities','Total Equity']).fillna(0).copy()
dow30_financial_data[:2]

ROE  DebtEquity  OpMargin  Fscore
Ticker Report Date                                        
AAPL   2000-06-30   0.000000     0.00000  0.092055     0.0
       2000-09-30   0.041393     0.65644  0.044920     0.0

In [16]:
dow30_financial_data.columns

Index(['ROE', 'DebtEquity', 'OpMargin', 'Fscore'], dtype='object')

In [17]:
# Custom method for reindexing
method = lambda x: x.interpolate(method='quadratic')

In [18]:
# Custom resampling method.
dow30_daily_derived = sf.reindex(df_src=dow30_financial_data, df_target=dow30_px_history,
                      group_index=TICKER, method=method)

dow30_reindex_derived = sf.reindex(df_src=dow30_daily_derived,
                       df_target=dow30_px_latest,
                       union=True, only_target_index=False,
                       method='ffill')
dow30_reindex_derived.groupby(TICKER).head(5), dow30_reindex_derived.groupby(TICKER).tail(5)

(                        ROE  DebtEquity  OpMargin    Fscore
 Ticker Date                                                
 AAPL   2007-01-03  0.094670    0.779770  0.186899  5.988432
        2007-01-04  0.094642    0.780957  0.187239  5.982827
        2007-01-05  0.094599    0.782153  0.187567  5.976349
        2007-01-08  0.094382    0.785789  0.188479  5.951670
        2007-01-09  0.094279    0.787017  0.188759  5.941696
 AMGN   2007-01-03       NaN         NaN       NaN       NaN
        2007-01-04       NaN         NaN       NaN       NaN
        2007-01-05       NaN         NaN       NaN       NaN
        2007-01-08       NaN         NaN       NaN       NaN
        2007-01-09       NaN         NaN       NaN       NaN
 AXP    2007-01-03       NaN         NaN       NaN       NaN
        2007-01-04       NaN         NaN       NaN       NaN
        2007-01-05       NaN         NaN       NaN       NaN
        2007-01-08       NaN         NaN       NaN       NaN
        2007-01-09      

In [20]:
signals_df = dow30_reindex_derived.copy()

In [21]:
fund_signal_df = signals_df.drop(columns = ['ROE', 'DebtEquity', 'OpMargin','Fscore'])

In [22]:
fund_signal_df.columns

Index([], dtype='object')

In [25]:
# Reference documentation for 'Return on Equity' threshold selection
'https://www.readyratios.com/reference/profitability/return_on_equity.html'

ROE_sig = np.where(dow30_reindex_derived['ROE'] >= 15.0, 1.0 , 0.0)
fund_signal_df['ROE_Sig'] = ROE_sig

# Reference documentation for 'Operating Margin' threshold selection
'https://www.readyratios.com/sec/ratio/ros/'

op_margin_sig = np.where(dow30_reindex_derived['OpMargin'] >= 10.0, 1.0 , 0.0)
fund_signal_df["OpMargin_Sig"] = op_margin_sig

# Reference documentation for 'Liabilities to Equity Ratio' threshold selection
'https://www.readyratios.com/sec/ratio/financial-leverage/'

debt_equity_sig = np.where(dow30_reindex_derived['DebtEquity'] <= 1, 1.0 , 0.0)
fund_signal_df["DebtEq_Sig"] = debt_equity_sig

# Reference documentation for 'Pietroski F-Score' threshold selection
'https://www.oldschoolvalue.com/investment-tools/piotroski-f-score-screening-early-often/'

f_score_sig = np.where(dow30_reindex_derived['Fscore'] >= 6, 1.0 , 0.0)
fund_signal_df["Fscore_Sig"] = f_score_sig

In [26]:
fund_signal_df

ROE_Sig  OpMargin_Sig  DebtEq_Sig  Fscore_Sig
Ticker Date                                                     
AAPL   2007-01-03      0.0           0.0         1.0         0.0
       2007-01-04      0.0           0.0         1.0         0.0
       2007-01-05      0.0           0.0         1.0         0.0
       2007-01-08      0.0           0.0         1.0         0.0
       2007-01-09      0.0           0.0         1.0         0.0
       2007-01-10      0.0           0.0         1.0         0.0
       2007-01-11      0.0           0.0         1.0         0.0
       2007-01-12      0.0           0.0         1.0         0.0
       2007-01-16      0.0           0.0         1.0         0.0
       2007-01-17      0.0           0.0         1.0         0.0
       2007-01-18      0.0           0.0         1.0         0.0
       2007-01-19      0.0           0.0         1.0         0.0
       2007-01-22      0.0           0.0         1.0         0.0
       2007-01-23      0.0           0.0         1.0         0.0
       2007-01-24      0.0           0.0         1.0         0.0
       2007-01-25      0.0           0.0         1.0         0.0
       2007-01-26      0.0           0.0         1.0         0.0
       2007-01-29      0.0           0.0         1.0         0.0
       2007-01-30      0.0           0.0         1.0         0.0
       2007-01-31      0.0           0.0         1.0         0.0
       2007-02-01      0.0           0.0         1.0         0.0
       2007-02-02      0.0           0.0         1.0         0.0
       2007-02-05      0.0           0.0         1.0         0.0
       2007-02-06      0.0           0.0         1.0         0.0
       2007-02-07      0.0           0.0         1.0         0.0
       2007-02-08      0.0           0.0         1.0         0.0
       2007-02-09      0.0           0.0         1.0         0.0
       2007-02-12      0.0           0.0         1.0         0.0
       2007-02-13      0.0           0.0         1.0         0.0
       2007-02-14      0.0           0.0         1.0         0.0
...                    ...           ...         ...         ...
WMT    2020-08-31      0.0           0.0         0.0         1.0
       2020-09-01      0.0           0.0         0.0         1.0
       2020-09-02      0.0           0.0         0.0         1.0
       2020-09-03      0.0           0.0         0.0         1.0
       2020-09-04      0.0           0.0         0.0         1.0
       2020-09-08      0.0           0.0         0.0         1.0
       2020-09-09      0.0           0.0         0.0         1.0
       2020-09-10      0.0           0.0         0.0         1.0
       2020-09-11      0.0           0.0         0.0         1.0
       2020-09-14      0.0           0.0         0.0         1.0
       2020-09-15      0.0           0.0         0.0         1.0
       2020-09-16      0.0           0.0         0.0         1.0
       2020-09-17      0.0           0.0         0.0         1.0
       2020-09-18      0.0           0.0         0.0         1.0
       2020-09-21      0.0           0.0         0.0         1.0
       2020-09-22      0.0           0.0         0.0         1.0
       2020-09-23      0.0           0.0         0.0         1.0
       2020-09-24      0.0           0.0         0.0         1.0
       2020-09-25      0.0           0.0         0.0         1.0
       2020-09-28      0.0           0.0         0.0         1.0
       2020-09-29      0.0           0.0         0.0         1.0
       2020-09-30      0.0           0.0         0.0         1.0
       2020-10-01      0.0           0.0         0.0         1.0
       2020-10-02      0.0           0.0         0.0         1.0
       2020-10-05      0.0           0.0         0.0         1.0
       2020-10-06      0.0           0.0         0.0         1.0
       2020-10-07      0.0           0.0         0.0         1.0
       2020-10-08      0.0           0.0         0.0         1.0
       2020-10-09      0.0           0.0         0.0  

In [ ]:
# dow30_reindex_derived.to_csv(r'dow30_fundamentals.csv')

In [ ]:
# fund_signal_df.to_csv(r'fundamental_signals.csv')